# Gauss-Legendre Quadrature

## Imports

In [ ]:
import numpy as np
from plotly import __version__
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
init_notebook_mode(connected=True)
np.set_printoptions(suppress=True)

## Helper functions

## Quick primer on Vandermonde matrices

## Legendre polynomials

Just as orthonormal vectors serve as a basis for some vector subspace, Legendre polynomials span the subspaces of polynomials.



<img src="img/legendre_polynomials.png" alt="legendre-polynomials" style="width: 75%"/>

## Resources

[Orthogonal Polynomials](http://web.cs.iastate.edu/~cs577/handouts/orthogonal-polys.pdf)